In [4]:
import pymongo
import pprint
from bson.json_util import dumps, loads

client = pymongo.MongoClient("localhost",27017)

db = client.sample_airbnb
coll = db.listingsAndReviews

def show_all(cursor):
    pprint.pprint([a for a in cursor])

# Exercises

/usr/share/databases/wcorning/mongo-monday.ipynb

Exercise: Use $match to find all apartments with more than two bedrooms.

Exercise: Use $project on this output to get the name, country code, and total room count.

Exercise: Use $group on this output to get the average number of total rooms per country code.

In [33]:
show_all(coll.aggregate([
    {
        '$match': {
            'weekly_price': {'$exists':True}
        }
    },
    {
        '$group': {
            '_id':  {'$round': [{'$divide': ['$weekly_price', 2000.0]},0]},
            'avg_bed': {'$avg': '$bedrooms'},
            'avg_price': {'$avg': '$weekly_price'},
        }
    },
    {
        '$project': {
            '_id': 0,
            'bracket': {'$convert': {'input':'$_id', 'to':'int' }},
            'avg_bed': 1,
            'avg_price': {'$round': ['$avg_price',2]}
        }
    },
    {
        '$sort': {'bracket': 1}
    }
]))

[{'avg_bed': 1.2155172413793103,
  'avg_price': Decimal128('571.37'),
  'bracket': 0},
 {'avg_bed': 1.4186046511627908,
  'avg_price': Decimal128('1439.72'),
  'bracket': 1},
 {'avg_bed': 1.8888888888888888,
  'avg_price': Decimal128('3840.00'),
  'bracket': 2},
 {'avg_bed': 1.0, 'avg_price': Decimal128('5956.00'), 'bracket': 3},
 {'avg_bed': 2.6666666666666665,
  'avg_price': Decimal128('7953.00'),
  'bracket': 4}]


In [35]:
coll.find_one()

{'_id': '13328702',
 'listing_url': 'https://www.airbnb.com/rooms/13328702',
 'name': 'Super Studio in Wailea, Maui.',
 'summary': "My place is close to Wailea, Maui  hotels, beaches and golf courses. You’ll love my place because it's a very inexpensive way to stay in the best place on earth. My place is good for couples, solo adventurers, and business travelers.",
 'space': '',
 'description': "My place is close to Wailea, Maui  hotels, beaches and golf courses. You’ll love my place because it's a very inexpensive way to stay in the best place on earth. My place is good for couples, solo adventurers, and business travelers.",
 'neighborhood_overview': '',
 'notes': '',
 'transit': '',
 'access': '',
 'interaction': '',
 'house_rules': '',
 'property_type': 'Condominium',
 'room_type': 'Entire home/apt',
 'bed_type': 'Real Bed',
 'minimum_nights': '2',
 'maximum_nights': '1125',
 'cancellation_policy': 'strict_14_with_grace_period',
 'last_scraped': datetime.datetime(2019, 3, 6, 5, 0),

In [47]:
coll.find_one(
    {'amenities': {'$not': {'$all': ['TV','Internet','Wifi']}}},
    {'amenities'}
) 

{'_id': '13382536',
 'amenities': ['TV',
  'Cable TV',
  'Wifi',
  'Air conditioning',
  'Kitchen',
  'Free parking on premises',
  'Elevator',
  'Heating',
  'Family/kid friendly',
  'Washer',
  'Dryer',
  'Smoke detector',
  'Carbon monoxide detector',
  'First aid kit',
  'Essentials',
  'Shampoo',
  'Hangers',
  'Hair dryer',
  'Iron',
  'Laptop friendly workspace',
  'Pack ’n Play/travel crib',
  'Body soap',
  'Bath towel',
  'Toilet paper',
  'Bed linens',
  'Extra pillows and blankets',
  'Coffee maker',
  'Dishwasher',
  'Oven',
  'Smart TV',
  'DVD player',
  'Beach view',
  'Rain shower',
  'Terrace',
  'Balcony',
  'Sound system',
  'Breakfast table',
  'Espresso machine',
  'Double oven',
  'Murphy bed',
  'Bedroom comforts',
  'Bathroom essentials']}

In [39]:
coll.find_one(
    {'reviews': {'$elemMatch': {'comments': {'$regex': 'dirty'}}} },
    {'reviews.comments'}
)

[{'_id': '13778742',
  'reviews': [{'comments': 'The apartment was great!! Even better than in the '
                           'pics. Communication was extra smooth with '
                           'Alexandra. We loved the location, walking distance '
                           'to the city centre of Porto. Parking and aircon '
                           'were awesome. Recommendations on restaurants were '
                           'wonderful, plenty of local choices very close to '
                           'the apartment. Safe area and comfortable bed. '
                           "Strongly recommended. With no doubt we'll stay "
                           'there when we go back to Porto. Obrigada!!!!'},
              {'comments': 'De las mejores experiencias con airbnb, el '
                           'apartamento esta genial, muy acogedor limpio y no '
                           'muy lejos del centro. \r\n'
                           '\r\n'
                           'Alexandra

Exception ignored in: <bound method Cursor.__del__ of <pymongo.cursor.Cursor object at 0x7f55e9910390>>
Traceback (most recent call last):
  File "/usr/local/lib64/python3.6/site-packages/pymongo/cursor.py", line 246, in __del__
    def __del__(self):
KeyboardInterrupt


'comments': 'Perfect location! Perfect condo, clean, spacious, '
                           'bright and had everything needed to feel like '
                           'home!! Jean Michel was welcoming and responsive. '
                           'Would definitely come back!!'},
              {'comments': 'Thank you for your highly rated warm concierge & '
                           'coordinating our arrival and throughout our '
                           'stay. \n'
                           '\n'
                           ' '},
              {'comments': 'Nice spacious condo in a great location!'},
              {'comments': 'Amazing! We are so sad to leave!!!! Waikiki is a '
                           "lot of fun and Jean-Michel's condo is in the "
                           'perfect location to experience it. Very walkable '
                           'to the beach, shops and restaurants, and public '
                           'transit. We have a one year old and the three of '
  

In [24]:
coll.find_one(
    {'bedrooms': 2},
    {'reviews','host.host_name', 'name'}
)
# SELECT host.host_name, name FROM coll WHERE bedrooms=2;

{'_id': '13382536',
 'name': 'Modern Beachside Apartment with Balcony',
 'host': {'host_name': 'David'},
 'reviews': [{'_id': '140499497',
   'date': datetime.datetime(2017, 3, 30, 4, 0),
   'listing_id': '13382536',
   'reviewer_id': '117161775',
   'reviewer_name': 'Darmo',
   'comments': "David's large, modern, cozy, clean and comfortable apartment is located in the newer section of Barcelona, in a quiet neighbourhood. The large underground parking is well lit, clean and roomy making manouvering our car very easy. It has direct elevator access to the apartment above, so unloading and loading the car was very convenient.\n\nThe area around the apartment has stores, restaurants, a Mercadona Supermarket, and many smaller grocery stores. The sidewalks are broad and evenly graded which makes for easy walking, and clean.\n\nPublic buses are frequent and easy to use, and there is an underground Metro station nearby within easy walking distance.\n\nDavid was what I would call a super host. 

In [25]:
show_all(
    coll.find(
        {'address.country_code': 'BR', 'bedrooms': 2},
        {'address.country_code': 1, 'bedrooms': 1}
    )
)

[{'_id': '10411341', 'address': {'country_code': 'BR'}, 'bedrooms': 2},
 {'_id': '10570041', 'address': {'country_code': 'BR'}, 'bedrooms': 2},
 {'_id': '10201975', 'address': {'country_code': 'BR'}, 'bedrooms': 2},
 {'_id': '10706900', 'address': {'country_code': 'BR'}, 'bedrooms': 2},
 {'_id': '12091894', 'address': {'country_code': 'BR'}, 'bedrooms': 2},
 {'_id': '12234922', 'address': {'country_code': 'BR'}, 'bedrooms': 2},
 {'_id': '17693637', 'address': {'country_code': 'BR'}, 'bedrooms': 2},
 {'_id': '1940174', 'address': {'country_code': 'BR'}, 'bedrooms': 2},
 {'_id': '20116519', 'address': {'country_code': 'BR'}, 'bedrooms': 2},
 {'_id': '20055406', 'address': {'country_code': 'BR'}, 'bedrooms': 2},
 {'_id': '15177216', 'address': {'country_code': 'BR'}, 'bedrooms': 2},
 {'_id': '20599393', 'address': {'country_code': 'BR'}, 'bedrooms': 2},
 {'_id': '21291633', 'address': {'country_code': 'BR'}, 'bedrooms': 2},
 {'_id': '22952561', 'address': {'country_code': 'BR'}, 'bedrooms

In [26]:
show_all(
    coll.find(
        {'address.country_code': 'BR'},
        {'name': 1, 'host.host_name': 1}
    )
)

[{'_id': '10635276',
  'host': {'host_name': 'Luiz Henrique'},
  'name': 'GREAT Apartment at Barra da Tijuca'},
 {'_id': '10038496',
  'host': {'host_name': 'Ana Valéria'},
  'name': 'Copacabana Apartment Posto 6'},
 {'_id': '13810527',
  'host': {'host_name': 'Paulo'},
  'name': 'HUMAITÁ - VIEW OF THE CRISTO REDENTOR STATUE'},
 {'_id': '10373872',
  'host': {'host_name': 'Marcelo'},
  'name': 'Apartamento Mobiliado - Lgo do Machado'},
 {'_id': '10411341',
  'host': {'host_name': 'Liliane'},
  'name': 'Beautiful flat with services'},
 {'_id': '10570041',
  'host': {'host_name': 'Roberta (Beta) Gatti'},
  'name': 'Jardim Botânico Gourmet 2 bdroom'},
 {'_id': '10201975',
  'host': {'host_name': 'Barbara'},
  'name': 'Ipanema: moderno apê 2BR + garagem'},
 {'_id': '15015363',
  'host': {'host_name': 'Vera'},
  'name': 'Achei! Loft  na praia, moderno e equipado.'},
 {'_id': '10706900',
  'host': {'host_name': 'Aline'},
  'name': 'Apart-hotel w/ Wonderful Ocean View - 2 bedrooms!!'},
 {'_id

In [27]:
coll.find_one()

{'_id': '13328702',
 'listing_url': 'https://www.airbnb.com/rooms/13328702',
 'name': 'Super Studio in Wailea, Maui.',
 'summary': "My place is close to Wailea, Maui  hotels, beaches and golf courses. You’ll love my place because it's a very inexpensive way to stay in the best place on earth. My place is good for couples, solo adventurers, and business travelers.",
 'space': '',
 'description': "My place is close to Wailea, Maui  hotels, beaches and golf courses. You’ll love my place because it's a very inexpensive way to stay in the best place on earth. My place is good for couples, solo adventurers, and business travelers.",
 'neighborhood_overview': '',
 'notes': '',
 'transit': '',
 'access': '',
 'interaction': '',
 'house_rules': '',
 'property_type': 'Condominium',
 'room_type': 'Entire home/apt',
 'bed_type': 'Real Bed',
 'minimum_nights': '2',
 'maximum_nights': '1125',
 'cancellation_policy': 'strict_14_with_grace_period',
 'last_scraped': datetime.datetime(2019, 3, 6, 5, 0),

In [28]:
coll.find_one()

{'_id': '13328702',
 'listing_url': 'https://www.airbnb.com/rooms/13328702',
 'name': 'Super Studio in Wailea, Maui.',
 'summary': "My place is close to Wailea, Maui  hotels, beaches and golf courses. You’ll love my place because it's a very inexpensive way to stay in the best place on earth. My place is good for couples, solo adventurers, and business travelers.",
 'space': '',
 'description': "My place is close to Wailea, Maui  hotels, beaches and golf courses. You’ll love my place because it's a very inexpensive way to stay in the best place on earth. My place is good for couples, solo adventurers, and business travelers.",
 'neighborhood_overview': '',
 'notes': '',
 'transit': '',
 'access': '',
 'interaction': '',
 'house_rules': '',
 'property_type': 'Condominium',
 'room_type': 'Entire home/apt',
 'bed_type': 'Real Bed',
 'minimum_nights': '2',
 'maximum_nights': '1125',
 'cancellation_policy': 'strict_14_with_grace_period',
 'last_scraped': datetime.datetime(2019, 3, 6, 5, 0),

In [29]:
def show_all(cursor):
    pprint.pprint([a for a in cursor])

In [30]:

show_all(coll.aggregate( [
  {
    '$group': {'_id': '$bedrooms', 'how_many': {'$sum': 1}, 'avg_bath': {'$avg': '$bathrooms'}}
  }
]))

[{'_id': 0,
  'avg_bath': Decimal128('1.012195121951219512195121951219512'),
  'how_many': 123},
 {'_id': 3,
  'avg_bath': Decimal128('2.020833333333333333333333333333333'),
  'how_many': 72},
 {'_id': 5, 'avg_bath': Decimal128('3.0'), 'how_many': 1},
 {'_id': 6, 'avg_bath': Decimal128('5.0'), 'how_many': 1},
 {'_id': 8, 'avg_bath': Decimal128('5.0'), 'how_many': 2},
 {'_id': 7, 'avg_bath': Decimal128('3.0'), 'how_many': 3},
 {'_id': 1,
  'avg_bath': Decimal128('1.087227414330218068535825545171340'),
  'how_many': 643},
 {'_id': 2,
  'avg_bath': Decimal128('1.479166666666666666666666666666667'),
  'how_many': 216},
 {'_id': 4,
  'avg_bath': Decimal128('2.793103448275862068965517241379310'),
  'how_many': 29}]


In [31]:
show_all(coll.aggregate([
    {
      '$group': {
        '_id': '$address.country_code',
        'bed': {'$avg': '$bedrooms'},
        'bath': {'$avg': '$bathrooms'},
      }
    },
    {
        '$project': {
            '_id': 0,
            'country': '$_id',
            'rooms': {'$add': ['$bed','$bath'] }
        }
    }
]))

[{'country': 'BR', 'rooms': Decimal128('2.744318181818181878739437706826720')},
 {'country': 'HK', 'rooms': Decimal128('2.277777777777777769553903521295137')},
 {'country': 'AU', 'rooms': Decimal128('2.608208955223880524104756591780765')},
 {'country': 'ES', 'rooms': Decimal128('2.608695652173912956591241551074706')},
 {'country': 'CA', 'rooms': Decimal128('2.579439252336448537950527637141047')},
 {'country': 'PT', 'rooms': Decimal128('2.649397590361445686824026779504493')},
 {'country': 'US', 'rooms': Decimal128('2.628787878787878812550410648235802')},
 {'country': 'TR', 'rooms': Decimal128('2.72')},
 {'country': 'CN', 'rooms': Decimal128('1.666666666666666629659232512494782')}]


# jsonSchema and validators

In [ ]:
target_schema = {
  "required": ["weekly_price"],
  "properties": {
    "address": {
      "bsonType": "object",
      "required": ["country_code"],
      "properties": {
        "country_code": {
          "$enum": ["US", "CA"]
        }
      }
    }
  }
}
coll.find_one({'$jsonSchema': target_schema})

In [ ]:
db = client.misc # switch database

In [ ]:
validator = {
    "$jsonSchema": {
      "bsonType": "object",
      "title": "Specify values for a person",
      "required": ["name","ssn"],
      "properties": {
        "ssn": {
          "bsonType": "string",
          "description": "SSN must be a string"
        },
        "dob": {
          "bsonType": "date",
          "description": "Date of birth, if known, must be represented by the `date` type"
        }
      }
    }
}

db.create_collection("people", validator=validator) # Require inserted data to match the validator's schema

In [ ]:
db.create_collection("numbers", validator={
  "$and": [
    {
    "$jsonSchema": {
      "required": ["x","y"],
      "properties": {
        "x": {"bsonType": "int"},
        "y": {"bsonType": "int"}
      }
    }
    },
    {
    "$expr": {
      "$lt": ["$x", "$y"]
    }}
  ]
})

In [ ]:
# This will throw an error!
db.numbers.insert_one({"x": 3, "y": 2})